# Playing with MPS
Start by copying the part of the code that transforms a generic state to an MPS from the sixth class

In [ ]:
import numpy as np 
import scipy.linalg as LA
def generate_random_state(N):
        dim_h =2**N
        init_state = np.zeros([dim_h,1])
        init_state[0]=1.
        random_h = np.array(np.random.rand(dim_h,dim_h)+1j*np.random.rand(dim_h,dim_h))
        random_h = random_h+random_h.T.conj()
        random_h = random_h/LA.norm(random_h)*N

        random_unitary =LA.expm(-1j*random_h)
        random_state=random_unitary@init_state
        return random_state
    
def compute_A_sigma_bulk(remaining_state,n,list_sigma):
        remaining_spins= len(remaining_state.shape)
        initial_shape =remaining_state.shape
        if n == 0 :
            #print('first')
            first_size =remaining_state.shape[0]
            second_size=np.prod(remaining_state.shape[1:])
        else:
            #print('bulk')
            first_size =remaining_state.shape[0]
            second_size=np.prod(remaining_state.shape[1:])
            remaining_state_c =remaining_state.reshape(first_size,second_size)
            first_size =np.prod(remaining_state.shape[0:2])
            second_size=np.prod(remaining_state.shape[2:])
            sigma_prev = list_sigma[n-1]
            remaining_state = sigma_prev@remaining_state_c
            remaining_state = remaining_state.reshape(initial_shape)
            
            
            
        remaining_state_matrix = remaining_state.reshape(
            first_size,second_size)
        
        A,sigma,rest = LA.svd(remaining_state_matrix,full_matrices
                             =False)
        
        chi=rest.shape[0]
        sigma =np.diag(sigma)
        if n == 0:
            rest_tensor =rest.reshape([rest.shape[0]]
                                  +list(remaining_state.shape[1:]))
        else:
            A =A.reshape(remaining_state.shape[0],remaining_state.shape[1],
                     A.shape[1])
            A = np.einsum('ij,jkl->ikl',LA.pinv(list_sigma[n-1]),A)
            rest_tensor =rest.reshape([rest.shape[0]]+
                                  list(remaining_state.shape[2:]))
        
        return A, sigma, rest_tensor
        
def state_reconstruct_einsum(list_A,list_sigma):
    N =len(list_A)
    avail_index = 'abcdefghlmnopqrstuvz'
    first_piece = list_A_tensors[0]
    for n in range(N-1):
        if n ==0:
            first_piece=np.einsum('ab,bd->ad',first_piece,list_sigma_tensors[n])
        else: 
            #print(first_piece.shape)
            #print(list_sigma_tensors[n].shape)
            num_left_indices=len(first_piece.shape)
            einsum_index=avail_index[0:num_left_indices]
            einsum_index+=','+avail_index[num_left_indices-1:num_left_indices+1]
            einsum_index+='->'+avail_index[0:num_left_indices-1]+avail_index[
               num_left_indices]
            #print(einsum_index)
            first_piece=np.einsum(einsum_index,
                                      first_piece,list_sigma_tensors[n])
        print('contracting '+str(n))
        num_left_indices=len(first_piece.shape)
        einsum_index=avail_index[0:num_left_indices]
        next_piece = list_A_tensors[n+1]
        other_index=len(next_piece.shape)
        einsum_index = einsum_index+','+avail_index[num_left_indices-1:
                    num_left_indices+other_index-1]
        einsum_index += '->'+avail_index[0:num_left_indices-1]+ avail_index[
            num_left_indices:num_left_indices+other_index-1]
        #print(einsum_index)
        first_piece =np.einsum(einsum_index,first_piece,next_piece)    
    return first_piece
def state_reconstruct_matmult(list_A,list_sigma):
    N =len(list_A)
    #print(N)
    list_d =[]
    list_d.append(list_A[0].shape[1])
    mat_left= list_A[0]
    for n in range(1,N-1):
        #print(n)
        mat_left = mat_left@list_sigma[n-1]
        mat_left = mat_left@list_A[n].reshape(list_A[n].shape[0],list_A[n].shape[1]*list_A[n].shape[2])
        mat_left = mat_left.reshape(mat_left.shape[0]*list_A[n].shape[1],list_A[n].shape[2])
        list_d.append(list_A[n].shape[1])
    
    mat_left = mat_left@list_sigma[N-2]
    mat_left = mat_left@list_A[N-1]
    list_d.append(list_A[N-1].shape[1])
    mat_left = mat_left.reshape(list_d)
    return mat_left


def create_MPS(N,state):
    remaining_state =state
    list_A_tensors =[]
    list_sigma_tensors =[]
    for n in range(N-1):
    #print(n)
        A,sigma,rest=compute_A_sigma_bulk(remaining_state,n,list_sigma_tensors)
    
        remaining_state =rest
    #print(rest.shape)
        list_A_tensors.append(A)
        list_sigma_tensors.append(sigma)

    
    list_A_tensors.append(rest)
    return list_A_tensors, list_sigma_tensors

Now let start generating some interesting states and obtain their MPS decomposition. Let's first generate the $W$ state:  
$$ |W  \rangle =\frac{1}{\sqrt{N}}\sum_{n=1}^{N} |0 \cdots 0 1_n 0\cdots 0\rangle  $$

Let' s transform it into an MPS using the function defined abvove

Let's have a look to the singular value of the $|W \rangle $ state

Now let's build the MPS for the state $WW$
$$ |W W \rangle =\frac{1}{\sqrt{N-1}}\sum_{n=1}^{N-1} |0 \cdots 0 1_n 1_{n+1} 0\cdots 0\rangle  $$

Now define a function that performs the truncation on the MPS you have obtained

# Matrix product operators

Now let's define a vector of operators, $v^i_{a,b} = \sigma^i_{a,b}$ where we select only a 2-dimensional subspace of the operator space, say $1, \sigma_x$

Now contract the first leg of the operator tensor with the physical leg of the MPS for the $WW$ state, what do you obtain?